# Sherrington-Kirkpatrick Model

The cost function of the SK model is
$$
\hat H_P = \frac{1}{\sqrt{N}}\sum_{i<j\leq N} J_{ij} \hat{Z}_i \hat{Z}_j,
$$
where the couplings $J_{ij}$ are i.i.d. standard Gaussian variables, i.e. with zero mean $\left\langle J_{ij} \right\rangle = 0$ and variance $ \left\langle J_{ij}^2 \right\rangle = J^2$.

In [ ]:
using DrWatson
@quickactivate "QAOA.jl"

include("../src/QAOA.jl")

using PyPlot
PyPlot.plt.style.use("paper.mplstyle")
using PyCall
np = pyimport("numpy");

In [ ]:
N = 4
σ2 = 1.
np.random.seed(1)
J = np.random.normal(0, σ2, size=(N, N)) ./ sqrt(N)
J = np.triu(J, k=1)
J = J + transpose(J)

In [ ]:
p = 2
problem = QAOA.Problem(p, zeros(N), J)

__Using the wrapper function:__

In [ ]:
SK_problem = QAOA.sherrington_kirkpatrick(N, σ2, num_layers=p, seed=137)

__Gradient optimization with [Zygote](https://fluxml.ai/Zygote.jl/latest/):__

In [ ]:
learning_rate = 0.02
cost, params, probs = QAOA.optimize_parameters(SK_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

__Optimization with [NLopt](https://nlopt.readthedocs.io/en/latest/):__

In [ ]:
cost, params, probs = QAOA.optimize_parameters(SK_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]), :LN_COBYLA)

In [ ]:
xlabels = []
for bstr in digits.(0:2^N-1, base=2, pad=N)
    push!(xlabels, "\$|" * prod([string(b) for b in bstr]) * "\\rangle\$")
end

figure(figsize=(5, 3.2))
ax = subplot(111)
bar(0:2^N-1, probs)
ax.set_xticks(0:2^N-1)
ax.set_xticklabels(xlabels, rotation=90)
minorticks_off()
tight_layout()